In [1]:
import pandas as pd
import numpy as np
df =np.array('churn-bigml-80.csv')
df = pd.read_csv('churn-bigml-80.csv')

In [ ]:
print(df.head())
print(df.tail())
print(df.info())
print(df.shape)
print(df.describe())

In [ ]:
print(df.isnull().sum())

In [ ]:
df.groupby('State')['Churn'].sum().sort_values(ascending=False)

In [ ]:
df['Account length'].mean()

In [ ]:
df.groupby('Area code')['International plan'].value_counts()

In [ ]:
df.groupby('Area code')['Voice mail plan'].value_counts()

In [8]:
df['24 hour charge'] = np.where( (df['Total day minutes'] > 0) |(df['Total eve charge'] > 0) | (df['Total night charge'] > 0),
    df['Total day charge'] + df['Total eve charge'] + df['Total night charge'],0
)

In [9]:
df['24 hour minutes'] = np.where((df['Total day minutes'] > 0) | (df['Total eve minutes'] > 0) | (df['Total night minutes'] > 0),
    df['Total day minutes'] +
    df['Total eve minutes'] +
    df['Total night minutes'], 0
)

In [10]:
df['24 hour calls'] = np.where(
    (df['Total day calls'] > 0) | (df['Total eve calls'] > 0) | (df['Total night calls'] > 0),
    df['Total day calls'] + df['Total eve calls'] + df['Total night calls'],
    0
)

In [ ]:
print(df['24 hour charge'].sum())
print(df['24 hour minutes'].mean())
print(df['24 hour calls'].mean())

In [12]:
df['charge per call'] = np.where(
    df['24 hour calls'] > 0,
    df['24 hour charge'] / df['24 hour calls'],
    0
)

In [13]:
df['charge per minute'] = np.where(
    df['24 hour minutes'] > 0,
    df['24 hour charge'] / df['24 hour minutes'],
    0
)

In [ ]:
df.groupby('Area code')['charge per call'].sum()

In [ ]:
df.groupby('Area code')['charge per minute'].sum()

In [ ]:
df.groupby('Area code')['24 hour calls'].sum()

In [ ]:
df.groupby('Area code')['24 hour minutes'].sum()

In [18]:
label = ['Low', 'Medium','High']
df['Usage group'] = pd.qcut(df['24 hour calls'], q = 3, labels=label)

In [ ]:
print(df['Usage group'])

In [ ]:
df[df['Churn'] == True].groupby('Usage group').size()

In [ ]:
df[df['Churn'] == False].groupby('Usage group').size()

In [ ]:
df.groupby('Churn')['charge per minute'].agg(mean_cost = 'mean',mid_cost = 'median', min_cost = 'min',  max_cost='max',customer_count='count').reset_index()

In [ ]:
df[df['Churn'] == False].groupby('Area code')['Total day calls'].size()

In [ ]:
df[df['Churn'] == False].groupby('Area code')['Total day calls'].size()

In [ ]:
df[df['Churn'] == False].groupby('Area code')['Total night calls'].size()

In [ ]:
label = ['Short', 'Average', 'Long']
df['account length group'] = pd.qcut(df['Account length'], q = 3, labels=label)
print(df['account length group'])

In [ ]:
bins = pd.qcut(df['Account length'], q=3)
print(bins.cat.categories)

In [ ]:
df[df['Churn'] == True].groupby('account length group').size()

In [ ]:
df[df['Churn'] == False].groupby('account length group').size()

In [30]:
df['total charge'] = np.where(
    (df['24 hour charge'] > 0) | (df['Total intl charge'] > 0),
    df['24 hour charge'] + df['Total intl charge'],
    0
)

In [31]:
df['total call'] = np.where(
    (df['24 hour calls'] > 0) | (df['Total intl calls'] > 0),
    df['24 hour calls'] + df['Total intl calls'],
    0
)

In [32]:
df['total minutes'] = np.where(
    (df['24 hour minutes'] > 0) | (df['Total intl minutes'] > 0),
    df['24 hour minutes'] + df['Total intl minutes'],
    0
)

In [ ]:
df.groupby('Churn')['total charge'].sum()

In [ ]:
df.groupby('Churn')['charge per minute'].mean()

In [ ]:
df.groupby('Churn')['International plan'].value_counts()

In [ ]:
df['unevenusage'] = (
    df['Total day minutes'] - df['Total night minutes']
).abs()
print(df['unevenusage'])

In [ ]:
df.groupby('Churn')['unevenusage'].describe()

In [38]:
print(df.columns)

Index(['State', 'Account length', 'Area code', 'International plan',
       'Voice mail plan', 'Number vmail messages', 'Total day minutes',
       'Total day calls', 'Total day charge', 'Total eve minutes',
       'Total eve calls', 'Total eve charge', 'Total night minutes',
       'Total night calls', 'Total night charge', 'Total intl minutes',
       'Total intl calls', 'Total intl charge', 'Customer service calls',
       'Churn', '24 hour charge', '24 hour minutes', '24 hour calls',
       'charge per call', 'charge per minute', 'Usage group',
       'account length group', 'total charge', 'total call', 'total minutes',
       'unevenusage'],
      dtype='object')


In [ ]:
df.groupby('Churn')['Voice mail plan'].value_counts()

In [42]:
df['top5'] = df['Total intl minutes'] >= df['Total intl minutes'].quantile(0.95)
df.groupby('top5')['Churn'].mean()

top5
False    0.143817
True     0.177778
Name: Churn, dtype: float64

In [ ]:
df['first'] = (
    df['Total day minutes'] - df['Total eve minutes']
).abs()
df['second'] = (
    df['Total eve minutes'] - df['Total night minutes']
).abs()
df['final'] = (
    df['first'] - df['second']
).abs()

df.groupby('Churn')[['first','second']].sum()
print(df['first'].head())
print(df['second'].head())
print(df['final'].head())

0     67.7
1     33.9
2    122.2
3    237.5
4     18.4
Name: first, dtype: float64
0     47.3
1     58.9
2     41.4
3    135.0
4     38.6
Name: second, dtype: float64
0     20.4
1     25.0
2     80.8
3    102.5
4     20.2
Name: final, dtype: float64


In [53]:
df.to_csv('final bigml.csv')